# Efficient Yelp API Calls - Core Assignment

**Student:** Matthew Malueg

# Imports and Loading

**Imports**

In [4]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

**API access**

In [2]:
# Load API credentials
with open('/Users/Malue/.secret/yelp_api.json') as f:
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

**Custom functions**

In [3]:
def create_json_file(JSON_FILE, delete_if_exists=False):

    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    # If file DOES exist:
    if file_exists == True:

        # Check if user wants to delete is exists
        if delete_is_exists==True:

            print(f'[!] {JSON_FILE} already exists. Deleting previous file...')
            # Delete file and confirm it no longer exists
            os.remove(JSON_FILE)
            # recursive call to function after old file deleted
            create_json_file(JSON_FILE, delete_if_exists=False)
        else:
            print(f'[i] {JSON_FILE} already exists.')

    # If file does NOT exist:
    else:

        # Inform user and save empty list
        print(f'[i] {JSON_FILE} not found. Saving empty list to new file.')

        ## Create any needed folders
        # Get folder name only
        folder = os.path.dirname(JSON_FILE)

        # If JSON_FILE included a folder
        if len(folder)>0:
            # Create the folder
            os.makedirs(folder, exist_ok=True)
        # Save empty list to start the json file
        with open(JSON_FILE, 'w') as f:
            json.dump([], f)

# Generating file and making the API call

**Define file and key terms for API call**

In [5]:
# Specify JSON_FILE filename
# Include search terms in the filename
JSON_FILE = 'Data/results_in_progress_SA_sushi.json'

# Set API call parameters
LOCATION = 'San Antonio, TX'
TERM = 'Sushi'

**Create the initial empty file and determine preliminary information**

In [6]:
# Create a new empty json file (explicitly delete the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
# Load previous results (should now be zero) and use len of results for offset
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)

# Set offset based on previous results
n_results = len(previous_results) # Use this for the offset
print(f'- {n_results} previous results found.')

## Performing the API call
# Use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                                offset=n_results)
# How many results in total?
total_results = results['total']
print(f'There were {total_results} total results found, including any duplicates.') 
# There should be zero results at this point, as we have a fresh file.

# How many results did we get details for in a single page?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total required number of pages of results
n_pages = math.ceil((results['total'] - n_results) / results_per_page)
print(f'We would need {n_pages} pages to display all results.')

[i] Data/results_in_progress_SA_sushi.json not found. Saving empty list to new file.
- 0 previous results found.
There were 284 total results found, including any duplicates.
We would need 15 pages to display all results.


**Populate the json file with additional pages of query results**

In [7]:
for i in tqdm_notebook(range(1, n_pages+1)):

    ## PROGRESS OF QUERY SO FAR
    # Read in results in progess file and check length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    # Save number of results to use as offset
    n_results = len(previous_results)
    # Stop query if it would cause an error
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break

    ## ADDING TO FILE WITH NEXT PAGE OF QUERY RESULTS
    # Use n_results as the offset
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM,
                                    offset=n_results)
    # Append new results and save to file
    previous_results.extend(results['businesses'])
    # Display previous_results
    with open(JSON_FILE, 'w') as f:
        json.dump(previous_results, f)

    time.sleep(0.1)

  0%|          | 0/15 [00:00<?, ?it/s]

# Final results

**Examine final DataFrame**

In [10]:
# Load in final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(2), final_df.tail(2))

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8aUAFZSENBgdC17hPpKp5g,wild-goji-restaurant-and-bar-san-antonio-3,Wild Goji Restaurant & Bar,https://s3-media1.fl.yelpcdn.com/bphoto/KApU2Q...,False,https://www.yelp.com/biz/wild-goji-restaurant-...,1195,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.5,"{'latitude': 29.52242, 'longitude': -98.50648}","[restaurant_reservation, delivery]",$$,"{'address1': '7115 Blanco Rd', 'address2': 'St...",+12102770638,(210) 277-0638,3231.701220
1,l7tVQWHt14p6fcGdUvx3xw,shiro-japanese-sushi-bistro-san-antonio,Shiro Japanese Sushi Bistro,https://s3-media2.fl.yelpcdn.com/bphoto/dJmKJW...,False,https://www.yelp.com/biz/shiro-japanese-sushi-...,209,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 29.4361425988757, 'longitude': -9...",[delivery],$$$,"{'address1': '107 W Jones Ave', 'address2': ''...",,,7240.257501


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
282,bcFjXrZZ_dbIoGZNVxmSXA,h-e-b-plus-san-antonio-14,H-E-B plus!,https://s3-media3.fl.yelpcdn.com/bphoto/1RW4hx...,False,https://www.yelp.com/biz/h-e-b-plus-san-antoni...,22,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.0,"{'latitude': 29.379990584309308, 'longitude': ...",[],$,"{'address1': '4100 S New Braunfels Ave', 'addr...",+12105313100,(210) 531-3100,13846.739121
283,zGljSEDub70vPTe0D7JWxw,mia-marcos-pizza-selma,Mia Marco's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/Og1WQw...,False,https://www.yelp.com/biz/mia-marcos-pizza-selm...,193,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.5,"{'latitude': 29.584521, 'longitude': -98.306215}",[delivery],$$,"{'address1': '15939 IH 35 N', 'address2': '', ...",+12106678441,(210) 667-8441,22493.871237


In [11]:
# Check for and remove any duplicates
final_df.duplicated(subset='id').sum()

0

**Save to .csv**

In [12]:
# Save the final results to a compressed csv
final_df.to_csv('Data/final_results_SA_sushi.csv.gz', compression='gzip', index=False)